<a href="https://colab.research.google.com/github/Rizukaf-id/DRPM-machine-learning/blob/main/DRPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://drive.google.com/uc?export=download&id=<fileId>" -O <fileName>
!wget "https://docs.google.com/spreadsheets/d/1U86a-fSYtEh-z_Nk2JtZL8qq5ZMcA5T7HHhmZeCSQUs/edit?usp=drive_link" -O <fileName>

In [ ]:
import pandas as pd

In [ ]:
drive_link = "https://drive.google.com/file/d/1fVMMa7kHwuyEsi0DcRsQM9TNTRVRKQS6/view?usp=drive_link"
file_id = drive_link.split('/')[-2]
datafile_url = f"https://drive.google.com/uc?export=download&id={file_id}"